# In Notebook 2 is an incomplete process of refining the features collected into a format to model with the panel dataframe. While I did clean a decent portion of the data scraped there are still more to refine, and of the ones we did, there is still the challenge of missing values. In this notebook we'll handle the NaNs. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

%matplotlib inline
import os

In [3]:
#to start like in 2, let's just manage one 
#dataset at a time
soybean_yield = pd.read_csv('../dat/SOYBEANS-AcreageYieldProductionandPrice-2019-11-21.csv')
fertilizers = pd.read_csv('../dat/fertilizers_cleaned.csv')

In [20]:
soybean_yield

,YEAR,LOCATION,STATE ANSI,ASD CODE,COUNTY ANSI,REFERENCE PERIOD,COMMODITY,AREA HARVESTED in ACRES,AREA PLANTED in ACRES,PRODUCTION in BU,YIELD in BU / ACRE,PRODUCTION in $,PRICE RECEIVED in $ / BU
0,1996,ALABAMA,01,,,YEAR,SOYBEANS,"305,000","320,000","10,370,000",34,"76,738,000",
1,1996,ALABAMA,01,,,MARKETING YEAR,SOYBEANS,,,,,,7.4
2,1996,ARKANSAS,05,,,YEAR,SOYBEANS,"3,500,000","3,550,000","112,000,000",32,"824,320,000",
3,1996,ARKANSAS,05,,,MARKETING YEAR,SOYBEANS,,,,,,7.36
4,1996,DELAWARE,10,,,YEAR,SOYBEANS,"217,000","220,000","7,595,000",35,"54,684,000",
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1443,2019,SOUTH DAKOTA,46,,,YEAR,SOYBEANS,"3,560,000","3,600,000","153,080,000",43,,
1444,2019,TENNESSEE,47,,,YEAR,SOYBEANS,"1,380,000","1,400,000","64,860,000",47,,
1445,2019,TEXAS,48,,,YEAR,SOYBEANS,"68,000","80,000","2,108,000",31,,
1446,2019,VIRGINIA,51,,,YEAR,SOYBEANS,"560,000","570,000","19,040,000",34,,


In [25]:
def make_numbers(cleaned_df_):
    cleaned_df = cleaned_df_.copy(deep=True)
    for col in cleaned_df.columns[7:]: #5 for soybean yield
        cleaned_df.fillna('-1',inplace=True)
        cleaned_df[col] = cleaned_df[col].apply(lambda z:
                       z.replace('(D)','0'))
        cleaned_df[col] = cleaned_df[col].apply(lambda z:
                       z.replace('(NA)','-1'))
        cleaned_df[col] = cleaned_df[col].apply(lambda z:
                       z.replace('(Z)','0'))

        cleaned_df[col] = cleaned_df[col].apply(lambda z:
                       z.replace(' ','0'))
        cleaned_df[col] = cleaned_df[col].apply(lambda z:
                       z.replace(',',''))
        cleaned_df[col] = cleaned_df[col].apply(lambda z:
                                               float(z))    
    
    return cleaned_df.replace(-1,np.nan)

In [26]:
soybean_yield = make_numbers(soybean_yield)

In [8]:
fertilizers.YEAR.nunique()

13

In [9]:
fertilizers.LOCATION.nunique()

22

In [10]:
soybean_yield.YEAR.nunique()

24

In [11]:
soybean_yield.LOCATION.nunique()

32

So what do we want to do? See what states and/or years they share. 

In [15]:
for loc in soybean_yield.LOCATION.unique():
    if loc not in fertilizers.LOCATION.unique():
        print(loc , 'will get tossed. ')

ALABAMA will get tossed. 
FLORIDA will get tossed. 
GEORGIA will get tossed. 
NEW JERSEY will get tossed. 
OKLAHOMA will get tossed. 
SOUTH CAROLINA will get tossed. 
TEXAS will get tossed. 
NEW YORK will get tossed. 
WEST VIRGINIA will get tossed. 
OTHER STATES will get tossed. 


These are states we have yield data for but no fertilizer data for. 

In [16]:
yield_fertilizer = soybean_yield.merge(fertilizers,on=['YEAR','LOCATION'])